<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Lab
# AI Database Assistant

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('penguins.csv')

In [20]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Convert CSV File to SQL Database in RAM

In [21]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [22]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [23]:
# Name the new database table Penguins
data = df.to_sql(name='Penguins',con=temp_db)

2023-10-14 22:25:53,523 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2023-10-14 22:25:53,523 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-14 22:25:53,524 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2023-10-14 22:25:53,524 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-14 22:25:53,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-14 22:25:53,528 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2023-10-14 22:25:53,528 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2023-10-14 22:25:53,530 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2023-10-14 22:25:53,530 INFO sqlalchemy.engine.Engine [no key 0.00081s] ()
2023-10-14 22:25:53,532 INFO sqlalchemy.engine.Engine COMMIT
2023-10-14 22:25:53,533 INFO sqlalchemy.engine.Engine BE

## Connect to SQL DataBase

Using SQL Alchemy we can establish a connection to this temporary database and query it for the results:

In [24]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * FROM Penguins WHERE island='Torgersen'"))

2023-10-14 22:25:54,197 INFO sqlalchemy.engine.Engine SELECT * FROM Penguins WHERE island='Torgersen'
2023-10-14 22:25:54,198 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ()


In [25]:
result.all()

[(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
 (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
 (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE'),
 (3, 'Adelie', 'Torgersen', None, None, None, None, None),
 (4, 'Adelie', 'Torgersen', 36.7, 19.3, 193.0, 3450.0, 'FEMALE'),
 (5, 'Adelie', 'Torgersen', 39.3, 20.6, 190.0, 3650.0, 'MALE'),
 (6, 'Adelie', 'Torgersen', 38.9, 17.8, 181.0, 3625.0, 'FEMALE'),
 (7, 'Adelie', 'Torgersen', 39.2, 19.6, 195.0, 4675.0, 'MALE'),
 (8, 'Adelie', 'Torgersen', 34.1, 18.1, 193.0, 3475.0, None),
 (9, 'Adelie', 'Torgersen', 42.0, 20.2, 190.0, 4250.0, None),
 (10, 'Adelie', 'Torgersen', 37.8, 17.1, 186.0, 3300.0, None),
 (11, 'Adelie', 'Torgersen', 37.8, 17.3, 180.0, 3700.0, None),
 (12, 'Adelie', 'Torgersen', 41.1, 17.6, 182.0, 3200.0, 'FEMALE'),
 (13, 'Adelie', 'Torgersen', 38.6, 21.2, 191.0, 3800.0, 'MALE'),
 (14, 'Adelie', 'Torgersen', 34.6, 21.1, 198.0, 4400.0, 'MALE'),
 (15, 'Adelie', 'Torgersen', 36.6, 17.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


## Using LLMs to Generate SQL from Natural Language

First, let's create a prefix we can add to our prompts that helps the model understand the table:

In [80]:
def create_prefix(query): 
    
    prefix=f"""
Return a SQL statement that answers the following query:
{query}

For a table called 'Penguins' with the following properties:
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 
Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query. 
""" 
    return prefix

In [81]:
def user_input():
    query = input("Ask a question about the database")
    return create_prefix(query)

In [82]:
prefix = user_input()

Ask a question about the database How many Penguins are female?


In [83]:
print(prefix)


Return a SQL statement that answers the following query:
How many Penguins are female?

For a table called 'Penguins' with the following properties:
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 
Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query. 



In [84]:
from vertexai.language_models import CodeGenerationModel

In [85]:
codegen_model = CodeGenerationModel.from_pretrained('code-bison')

In [86]:
print("Hello! I am your AI Assistant. Ask me anything about the database")
print("\n\n")
prefix = user_input()
result = codegen_model.predict(prefix=prefix)

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database How many Penguins are female?


In [87]:
result

```sql
SELECT COUNT(*)
FROM Penguins
WHERE sex = 'FEMALE';
```

In [88]:
result.text

"```sql\nSELECT COUNT(*)\nFROM Penguins\nWHERE sex = 'FEMALE';\n```"

### Clean Text

In [89]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace("```",'').replace('\n',' ')

In [90]:
clean_sql(result.text)

" SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE'; "

### Combine everything together

In [93]:
def nlp_assistant():
    print("Hello! I am your AI Assistant. Ask me anything about the database")
    print("\n\n")
    prefix = user_input()
    result = codegen_model.predict(prefix=prefix)
    sql = clean_sql(result.text)
    with temp_db.connect() as conn:
        print("\nOk, I am running this SQL Statement")
        result = conn.execute(text(sql))
    print('-------')
    print('-------')
    print("Here is the answer to your query")
    print('\n')
    print(result.all())

In [94]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database How many Penguins are female?



Ok, I am running this SQL Statement
2023-10-14 22:44:07,798 INFO sqlalchemy.engine.Engine  SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE'; 
2023-10-14 22:44:07,800 INFO sqlalchemy.engine.Engine [cached since 26.29s ago] ()
-------
-------
Here is the answer to your query


[(165,)]


In [95]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database What are the unique island names?



Ok, I am running this SQL Statement
2023-10-14 22:44:16,000 INFO sqlalchemy.engine.Engine  SELECT DISTINCT island FROM Penguins; 
2023-10-14 22:44:16,003 INFO sqlalchemy.engine.Engine [generated in 0.00296s] ()
-------
-------
Here is the answer to your query


[('Torgersen',), ('Biscoe',), ('Dream',)]
